In [1]:
from utils import *

In [5]:
batch_size = 128
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'mps'

dataset = MNISTtrainer(batch_size)
params = dict(device=device,
                width=50, lr=0.01, num_train=10,
                sb=1, depth=5, sw=1.5)

model = generate_fc_dnn(dataset.input_dim, dataset.output_dim,
                params['depth'], params['width']).to(device)

pretrain_acc, pre_trained_model = compute_training_acc_epochs(model, dataset, params, debug=True)

Sequential(
  (0): Linear(in_features=784, out_features=50, bias=True)
  (1): Tanh()
  (2): Linear(in_features=50, out_features=50, bias=True)
  (3): Tanh()
  (4): Linear(in_features=50, out_features=50, bias=True)
  (5): Tanh()
  (6): Linear(in_features=50, out_features=50, bias=True)
  (7): Tanh()
  (8): Linear(in_features=50, out_features=10, bias=True)
  (9): LogSoftmax(dim=1)
) SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.01
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

Train set: Average loss: 0.9528, Accuracy: 42467.0/60000 (71%)

Epoch: 0 	Training Accuracy: 70.78%

Train set: Average loss: 0.6307, Accuracy: 48435.0/60000 (81%)

Epoch: 1 	Training Accuracy: 80.73%

Train set: Average loss: 0.5091, Accuracy: 50608.0/60000 (84%)

Epoch: 2 	Training Accuracy: 84.35%

Train set: Average loss: 0.4417, Accuracy: 51911.0/60000 (87%)

Epoch: 3 	Training Accuracy: 86.52%

Train set: Average loss: 0.3965, 

In [ ]:
dataset = MNISTtrainer(args.batch_size)
params = dict(device='cpu' if args.no_cuda else 'cuda',
                width=args.width, lr=args.lr, num_train=args.num_train,
                sb=0.05,
    )
if args.debug: print(params)

# run experiment in grid coordinate.
sws = np.linspace(1., 4., args.num_sw)

# test a lot less depths
depths = np.linspace(10, 100, args.num_depth, dtype=int)

pretrain_accs = list()
all_cut_accuracies = list()

for i, depth in enumerate(depths):
    for j, sw in enumerate(sws):
        params['depth'], params['sw'] = depth, sw
        
        # build and initialize the DNN model and optimizer
        model = generate_fc_dnn(dataset.input_dim, dataset.output_dim,
                        params['depth'], params['width']).to(device)
        
        pretrain_acc, pre_trained_model = compute_training_acc(model, dataset, params, debug=False)
        cut_accuracies = calculate_cut_accuracies(model, sw, sb, dataset, params)
        all_cut_accuracies.append(cut_accuracies)
        
        acc = compute_training_acc(dataset, params, debug=args.debug_train_acc)
        pretrain_accs.append(pretrain_acc)
        
        if args.debug: print('({},{})->[d={},sw={}]: \t Train Acc: {:.6f}'.format(i,j,depth,sw,acc))

# acc = np.array(accs).reshape((len(depths), len(sws)))
# np.savez('train_acc', xgrid=sws,ygrid=depths,train_acc=acc)